# Classification according the Ne II, Ne III and Ne V emission lines information - Download the Photometry Tables for each object found

From the objects that we find in NED with Ne emission lines, now we are going to classify according to what information the Ne II, Ne III and Ne V emission lines each have.

Also inside the main loop, for each object found in NED with Ne emission lines, we will save the photometry tables in a .csv file.

In [2]:
from datetime import date
import time
import pandas as pd
import numpy as np
import csv

from pathlib import Path
from astropy.table import Table

import astropy.units as u
from astropy.table import hstack

In [3]:
path_raw = '../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object

In [6]:
date_sample = 'Apr_06_2022'

In [14]:
NeIR_obj_t = Table.read(path_raw+'Obj_Ne-IR_'+date_sample+'.vot',format='votable')
NeIR_obj_t['Ne_IR_info'] = np.array(['Empty']*len(NeIR_obj_t),dtype='object') #Add a column to save the Ne info
NeIR_obj_t[356]

Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index,Ne_IR_info
object,object,float64,float64,object,float64,str1,object,str93,float64,object
Mrk 622,Mrk 622,121.92081574901,39.00423528003,Sy2,0.02335,z,AGN|G |IR |LIN|Rad|Sy2|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=121.92081574901%2039.00423528003&-c.rs=5,43765.0,Empty


The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which are related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Fq** we define a list with the those frequency values

In [11]:
Ne_IR_Fq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

In [25]:
i = 355
ned_table = Table.read(path_phot_ned+NeIR_obj_t['Main_Id'][i]+'_NED_phot_tables'+'.vot',
                           format='votable')

In [26]:
len(ned_table)

59

In [27]:
a=np.where(ned_table['Flux_Density'].mask)[0]
ned_table.remove_rows(np.where(ned_table['NED_Uncertainty'] == '')[0])
ned_table.remove_rows(np.where(ned_table['NED_Uncertainty'] == '+/-...')[0])
len(ned_table)

44

In [28]:
a

array([32, 33, 38, 41, 42, 45, 52, 53, 54, 55])

In [24]:
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
cds_table = Table.read(path_phot_cds+NeIR_obj_t['Main_Id'][i]+'_CDS_phot_tables'+'.vot',
                           format='votable')
l_cds = np.where(cds_table['Bibcode'] == '2011ApJ...740...94D')[0]
l_cds

array([], dtype=int64)

In [9]:
NeII_NeIII_NeV_IR_obj = [] #List of objects with NeII, NeIII and NeV data
NeII_NeIII_IR_obj = [] #List of objects with NeII and NeIII data
NeII_NeV_IR_obj = [] #List of objects with NeII and NeV data
NeIII_NeV_IR_obj = [] #List of objects with NeIII and NeV data
NeIII_IR_obj = [] #List of objects with NeIII data
NeV_IR_obj = [] #List of objects with NeV data
NeII_IR_obj = [] #List of objects with NeII data

#We create and open a file .csv to write the list of objects names with Ne_IR data
outfile = open(path_raw+'Obj_Ne-IR_v2_'+date_samples+'.csv','w')
writer = csv.writer(outfile) # create the csv writer
header = ['Name_Id','id_from','RA','DEC','otype_txt','redshift','rvz_type','otypes','Ne_IR_info','SMB_sample']#Creating header
writer.writerow(header) #Writing the header

start = time.time()
for ind, name in enumerate(NeIR_obj_t['Main_Id']):
    phot_t = Table.read(path_phot_ned+name+'_NED_phot_tables'+'.vot', format='votable')
    
    Fr=phot_t['Frequency'] #We get the Frequency column from the table
   
    if (Ne_IR_Fq[0] in Fr or Ne_IR_Fq[1] in Fr) and (Ne_IR_Fq[2] in Fr or Ne_IR_Fq[3] in Fr) and Ne_IR_Fq[4] in Fr:
        print(f"{ind}, {name}, NeII, NeIII and NeV")
        #Writing the object names
        outstring = [name,df['Id_used_NED'][ind],df['RA'][ind],df['DEC'][ind],df['otype_txt'][ind],df['redshift'][ind],
                     df['rvz_type'][ind],df['otypes'][ind],'NeII|NeIII|NeV',df['SMB_sample'][ind]]
        writer.writerow(outstring) #write the data
        NeII_NeIII_NeV_IR_obj.append(name) #Save the object name in the list NeII_NeIII_NeV_IR_obj
    
    elif (Ne_IR_Fq[0] in Fr or Ne_IR_Fq[1] in Fr) and Ne_IR_Fq[4] in Fr:
        print(f"{ind}, {name}, NeII and NeIII")
        #Writing the object names
        outstring = [name,df['Id_used_NED'][ind],df['RA'][ind],df['DEC'][ind],df['otype_txt'][ind],df['redshift'][ind],
                     df['rvz_type'][ind],df['otypes'][ind],'NeII|NeIII',df['SMB_sample'][ind]]
        writer.writerow(outstring) #write the data
        NeII_NeIII_IR_obj.append(name) #Save the object name in the list NeII_NeIII_IR_obj
    
    elif (Ne_IR_Fq[2] in Fr or Ne_IR_Fq[3] in Fr) and Ne_IR_Fq[4] in Fr:
        print(f"{ind}, {name}, NeII and NeV")
        #Writing the object names
        outstring = [name,df['Id_used_NED'][ind],df['RA'][ind],df['DEC'][ind],df['otype_txt'][ind],df['redshift'][ind],
                     df['rvz_type'][ind],df['otypes'][ind],'NeII|NeV',df['SMB_sample'][ind]]
        writer.writerow(outstring) #write the data
        NeII_NeV_IR_obj.append(name) #Save the object name in the list NeII_NeV_IR_obj
    
    elif (Ne_IR_Fq[0] in Fr or Ne_IR_Fq[1] in Fr) and (Ne_IR_Fq[2] in Fr or Ne_IR_Fq[3] in Fr):
        print(f"{ind}, {name}, NeIII and NeV")
        #Writing the object names
        outstring = [name,df['Id_used_NED'][ind],df['RA'][ind],df['DEC'][ind],df['otype_txt'][ind],df['redshift'][ind],
                     df['rvz_type'][ind],df['otypes'][ind],'NeIII|NeV',df['SMB_sample'][ind]]
        writer.writerow(outstring) #write the data
        NeIII_NeV_IR_obj.append(name) #Save the object name in the list NeIII_NeV_IR_obj
    
    elif Ne_IR_Fq[0] in Fr or Ne_IR_Fq[1] in Fr:
        print(f"{ind}, {name}, NeIII")
        #Writing the object names
        outstring = [name,df['Id_used_NED'][ind],df['RA'][ind],df['DEC'][ind],df['otype_txt'][ind],df['redshift'][ind],
                     df['rvz_type'][ind],df['otypes'][ind],'NeIII',df['SMB_sample'][ind]]
        writer.writerow(outstring) #write the data
        NeIII_IR_obj.append(name) #Save the object name in the list NeIII_IR_obj
    
    elif Ne_IR_Fq[2] in Fr or Ne_IR_Fq[3] in Fr:
        print(f"{ind}, {name}, NeV")
        #Writing the object names
        outstring = [name,df['Id_used_NED'][ind],df['RA'][ind],df['DEC'][ind],df['otype_txt'][ind],df['redshift'][ind],
                     df['rvz_type'][ind],df['otypes'][ind],'NeV',df['SMB_sample'][ind]]
        writer.writerow(outstring) #write the data
        NeV_IR_obj.append(name) #Save the object name in the list NeV_IR_obj
   
    elif Ne_IR_Fq[4] in Fr:
        print(f"{ind}, {name}, NeII")
        #Writing the object names
        outstring = [name,df['Id_used_NED'][ind],df['RA'][ind],df['DEC'][ind],df['otype_txt'][ind],df['redshift'][ind],
                     df['rvz_type'][ind],df['otypes'][ind],'NeII',df['SMB_sample'][ind]]
        writer.writerow(outstring) #write the data
        NeII_IR_obj.append(name) #Save the object name in the list NeII_IR_obj

end = time.time()

#we close the file with objects names
outfile.close()

print('\nTotal objects found with NeII, NeIII and NeV emission lines: ',len(NeII_NeIII_NeV_IR_obj))
print('Total objects found with NeII and NeIII emission lines: ',len(NeII_NeIII_IR_obj))
print('Total objects found with NeII and NeV emission lines: ',len(NeII_NeV_IR_obj))
print('Total objects found with NeIII and NeV emission lines: ',len(NeIII_NeV_IR_obj))
print('Total objects found with NeIII emission lines: ',len(NeIII_IR_obj))
print('Total objects found with NeV emission lines: ',len(NeV_IR_obj))
print('Total objects found with NeII emission lines: ',len(NeII_IR_obj))
print('\nExecution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

0, 3C 234.0, NeII, NeIII and NeV


NameError: name 'df' is not defined

In [18]:
df2 = pd.read_csv(path_raw+'Obj_Ne-IR_v2_'+date_samples+'.csv')
df2

,Name_Id,id_from,RA,DEC,otype_txt,redshift,rvz_type,otypes,Ne_IR_info,SMB_sample
0,3C 234.0,3C 234.0,150.456345,28.785818,Sy1,0.18494,z,AGN|G |IR |QSO|Rad|Sy1|X |blu|gam,NeII|NeIII|NeV,283
1,M 84,M 84,186.265597,12.886983,Sy2,0.00327,z,* |AGN|G |GiC|GiG|IR |Rad|Sy2|UV |X |smm,NeV,414
2,2MASS J13000535+1632148,NVSS J130005+163212,195.022283,16.537433,Sy1,0.07993,z,AGN|G |IR |Rad|Sy1|X,NeII|NeIII|NeV,428
3,NGC 262,NGC 262,12.196426,31.956957,Sy2,0.015260233939595302,v,* |AGN|Bla|G |GiG|IR |QSO|Rad|Sy2|X |gam,NeII|NeIII|NeV,484
4,IRAS 04385-0828,IRAS 04385-0828,70.229004,-8.372813,Sy2,0.015064,z,* |AGN|G |GiG|IR |Sy2,NeII|NeIII|NeV,541
...,...,...,...,...,...,...,...,...,...,...
354,NGC 3982,NGC 3982,179.117269,55.125239,Sy1,0.00371,z,AGN|G |GiG|HI |IR |Rad|Sy1|X,NeII|NeIII|NeV,43595
355,PSCz Q18368+3549,IRAS 18368+3549,279.647646,35.872153,Sy2,0.11617,z,AGN|G |IR |Mas|Sy2,NeII|NeIII|NeV,43670
356,Mrk 622,Mrk 622,121.920816,39.004235,Sy2,0.02335,z,AGN|G |IR |LIN|Rad|Sy2|X,NeII|NeIII|NeV,43765
357,[LGE2006] 12,[LGE2006] 12,163.000458,57.406528,Sy1,0.974,z,AGN|G |IR |Rad|Sy1|X |smm,NeIII,43819


### Watermark

In [12]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.31.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.13.0-30-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.0

astropy: 5.0
pandas : 1.4.1
numpy  : 1.20.3
csv    : 1.0

Watermark: 2.3.0

